# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import math
import re

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [20]:
# names us

colonne = ["year","name","gender","births"]

def df_names_us():
    
    #On commence par récupérer les noms des fichiers qui nous intéressent à l'aide d'os
    path = 'C:/Users/Jeanl/data/telecom_paris/700_python/Kit-Big-Data/session3/names/'
    fichiers = os.listdir(path)
    df_totale = []
    
    #On boucle sur les fichiers récupéres pour mettre en place la dataframe
    for files in fichiers:
        df_tmp = pd.read_csv(path + files, names=["name", "gender", "births"])
        df_tmp.insert(0, 'year', re.findall("[0-9]+", files)[0])
        df_totale.append(df_tmp)
    #On finit par concat à la suite nos df temporaires 
    df = pd.concat(df_totale).sort_values(by=["year", "gender", "births", "name"], ascending=[True, True, False, True])
    df.reset_index(drop=True, inplace=True)
    
    return df

In [21]:
data = df_names_us()

data

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


In [22]:
path = 'C:/Users/Jeanl/data/telecom_paris/700_python/Kit-Big-Data/session3/names'
fichiers = os.listdir(path)
#fichiers

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [23]:
# names fr
def df_names_fr():
    
    df=pd.read_csv('C:/Users/Jeanl/data/telecom_paris/700_python/Kit-Big-Data/session3/nat2021_csv/nat2021.csv',sep=';')
    
    #colonnes
    df=df.rename(columns={'sexe':'gender','preusuel':'name','annais':'year','nombre':'births'})
    df=df[['year','name','gender','births']]
    df['year']=df['year'].astype(str)
    df['name']=df['name'].astype(str)
    
    #gender
    df['gender']=df['gender'].replace(1,'M')
    df['gender']=df['gender'].replace(2,'F')
    
    #année
    df['year']=df['year'].astype(str)
    df=df.loc[df['year'].str.contains(r'\d{4}')]
    #Le name
    df['name']=df['name'].astype(str)
    df.drop(df.loc[df['name'] == '_PRENOMS_RARES'].index, inplace=True)
    df.drop(df.loc[df['name'].str.len() < 2].index, inplace=True)
    df['name'] = df['name'].apply(lambda name: ''.join(re.split(r'[^a-zA-Z ]+', name))) 
    df['name']=df['name'].str.title()
    

    #On trie à l'identique
    df=df.sort_values(by=['name','births','gender','year'],ascending=[True,False,True,True])
    df=df.reset_index(drop=True)

    return df

In [24]:
data_2 = df_names_fr()
#data_2

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [27]:
# taux de change
def df_taux_change(devises):
    
    
    df = pd.read_csv('C:/Users/Jeanl/data/telecom_paris/700_python/Kit-Big-Data/session3/Webstat_Export.csv',sep=";",low_memory=False)
    #On enlève les colonnes inutiles
    df.drop(df.head(6).index, inplace=True)
    
    #On met en place le format demandé pr l'index 
    df.rename(columns = {'Titre :':'date'}, inplace = True)
    df = df.set_index(pd.to_datetime(df["date"],infer_datetime_format=True))
    df = df.drop(columns = "date")
    
    df = df.sort_values(by=["date"], ascending=[True])
    df.drop(df.head(1).index, inplace=True)
    
    for colonne in df.columns.values:
        findall = re.findall('\((.*?)\)', colonne)
        if findall!=[]:
            df.rename(columns={str(colonne):str(findall[0])}, inplace = True)
            
    #On nettoie la base de données   
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.apply(lambda x: x.replace('-',None))
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce') )
    df = df.loc[:, ~df.columns.duplicated()]
    df = df.dropna(axis=0,thresh=10)
    
    
    return df[devises]




In [28]:

devises=["CHF", "GBP", "USD"]
data_3 = df_taux_change(devises)
data_3

,CHF,GBP,USD
date,,,
1999-01-05,1.6123,0.71220,1.1790
1999-01-06,1.6116,0.70760,1.1743
1999-01-07,1.6165,0.70585,1.1632
1999-01-08,1.6138,0.70940,1.1659
1999-01-09,1.6138,0.70940,1.1659
...,...,...,...
2022-09-15,0.9572,0.86934,0.9992
2022-09-16,0.9579,0.87400,0.9954
2022-09-17,0.9579,0.87400,0.9954


### Tests

In [29]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [30]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [31]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 20.532s

OK
